In [16]:
import numpy as np
import scipy.stats as ss
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
import tensorflow as tf
import tensorflow.experimental.numpy as tnp
import tensorflow_probability as tfp
import scipy.special

tfd = tfp.distributions
np.random.seed(0)

from LSM_tensorflow import *

In [17]:
job_name = 1

In [18]:
S0 = 27479.363
sigma = .1797
_lambda = 11.9488

eta1 = 1/0.074
eta2 = 1/0.0608
p = .3316
r = 0.01
d = 0.2

n = 200_000 # sample size
m = 360 # sample length
T = 1 #years

LTV0 = 0.6
LTVH = 0.8
liquidation_penalty = 0.95

degree = 2
seed = 0
print(n, m, LTV0, LTV0/LTVH)

200000 360 0.6 0.7499999999999999


In [19]:
LTVH = 0.9

start = -0.1
stop  = 0.8
step  = .05
all_d_arr = np.arange(start, stop+step, step)

start = 0.1
stop  = 0.8
step  = .05
LTV0_arr = np.arange(start, stop+step, step)

start = 10
stop  = m
step  = 10
slice_id_arr = np.arange(start, stop+step, step)

print(LTVH, all_d_arr, LTV0_arr, slice_id_arr)

0.9 [-0.1  -0.05  0.    0.05  0.1   0.15  0.2   0.25  0.3   0.35  0.4   0.45
  0.5   0.55  0.6   0.65  0.7   0.75  0.8 ] [0.1  0.15 0.2  0.25 0.3  0.35 0.4  0.45 0.5  0.55 0.6  0.65 0.7  0.75
 0.8  0.85] [ 10  20  30  40  50  60  70  80  90 100 110 120 130 140 150 160 170 180
 190 200 210 220 230 240 250 260 270 280 290 300 310 320 330 340 350 360]


In [20]:
job1 = all_d_arr[:7]
job2 = all_d_arr[7:14]
job3 = all_d_arr[14:]
job1, job2, job3

(array([-0.1 , -0.05,  0.  ,  0.05,  0.1 ,  0.15,  0.2 ]),
 array([0.25, 0.3 , 0.35, 0.4 , 0.45, 0.5 , 0.55]),
 array([0.6 , 0.65, 0.7 , 0.75, 0.8 ]))

In [29]:
results_filename = 'results%i.pickle'%job_name
payoffs_filename = 'payoffs%i.pickle'%job_name
if job_name == 1:
    d_arr = job1
elif job_name == 2:
    d_arr = job2
else:
    d_arr = job3


In [27]:
import pickle

# with open(r"all_results.pickle", "rb") as input_file:
#     all_results = pickle.load(input_file)
all_results = []
payoffs_results = []


In [28]:
finished_jobs = []

for a in all_results:
    finished_jobs.append((a[0],a[1],a[2]))

In [30]:
print(d_arr)

[-0.1  -0.05  0.    0.05  0.1   0.15  0.2 ]


In [8]:
_Kou_model_params = Kou_model_params(S0, sigma, _lambda, eta1, eta2, p, r, T)

for d in d_arr:
    _price_paths_simulation = price_paths_simulation(seed, n, m, _Kou_model_params, d)
    _contract_params =  contract_params(_price_paths_simulation.d, LTV0, LTVH, _Kou_model_params, liquidation_penalty)
    _price_paths_simulation.gen_price_paths()

    for LTV0 in LTV0_arr:
        _contract_params_job = contract_params(_price_paths_simulation.d, LTV0, LTVH, _Kou_model_params, _contract_params.liquidation_penalty)

        payoffs, healthy_mask = gen_payoffs_and_healthy_mask(_contract_params_job, _price_paths_simulation.price_t,
                                                            _price_paths_simulation.path_mins)
        
        payoffs_results.append([d, LTV0, payoffs, healthy_mask])
        
        with open(payoffs_filename, 'wb') as handle:
            pickle.dump(payoffs_results, handle)

        _LSM_regression = LSM_regression(_price_paths_simulation, 
                                        payoffs,
                                        _price_paths_simulation.price_t, 
                                        healthy_mask, degree, _contract_params)

        for slice_id in slice_id_arr:
            if (d, LTV0, slice_id) in finished_jobs:
                continue
            try:
                _LSM_regression.run_regression(slice_id, 1, False)
                
                _mean = _LSM_regression.A.mean()
                _var  = _LSM_regression.A.var()
                
                all_results.append([d, LTV0,  slice_id, _mean, _var])
            except:
                all_results.append([d, LTV0,  slice_id, 'error', 'error'])
                
            print('%.2f %.2f %i'%(d, LTV0,  slice_id))
            
        with open('results_filename.pickle', 'wb') as handle:
            pickle.dump(all_results, handle)
            print('saved')

Price paths (price_t) and running minimums (path_mins) are generated.
-0.10 0.10 10
-0.10 0.10 20
-0.10 0.10 30
-0.10 0.10 40
-0.10 0.10 50
-0.10 0.10 60
-0.10 0.10 70
-0.10 0.10 80
-0.10 0.10 90
-0.10 0.10 100
-0.10 0.10 110
-0.10 0.10 120
-0.10 0.10 130
-0.10 0.10 140
-0.10 0.10 150
-0.10 0.10 160
-0.10 0.10 170
-0.10 0.10 180
-0.10 0.10 190
-0.10 0.10 200
-0.10 0.10 210
-0.10 0.10 220
-0.10 0.10 230
-0.10 0.10 240
-0.10 0.10 250
-0.10 0.10 260
-0.10 0.10 270
-0.10 0.10 280
-0.10 0.10 290
-0.10 0.10 300
-0.10 0.10 310
-0.10 0.10 320
-0.10 0.10 330
-0.10 0.10 340
-0.10 0.10 350
-0.10 0.10 360
saved
-0.10 0.15 10
-0.10 0.15 20
-0.10 0.15 30
-0.10 0.15 40
-0.10 0.15 50
-0.10 0.15 60
-0.10 0.15 70
-0.10 0.15 80
-0.10 0.15 90
-0.10 0.15 100
-0.10 0.15 110
-0.10 0.15 120
-0.10 0.15 130
-0.10 0.15 140
-0.10 0.15 150
-0.10 0.15 160
-0.10 0.15 170
-0.10 0.15 180
-0.10 0.15 190
-0.10 0.15 200
-0.10 0.15 210
-0.10 0.15 220
-0.10 0.15 230
-0.10 0.15 240
-0.10 0.15 250
-0.10 0.15 260
-0.10 0.15 2